# Exercice 1
### Fourier-Motzkin

On considère un problème de minimisation sous forme standard :

\begin{cases} 
    \text{min } f(x) = \sum_{j=1}^{n}c_jx_j = c^Tx\\
    \sum_{j=1}^{n}a_{ij} \leq b_i, i=1,...,r \iff Ax \leq b \\
    x_j \geq 0, j=1,...,n \iff x\geq 0
\end{cases}

où $x=(x_1,...,x_n)^T \in \mathbb{R}^n, c=(c_1,...,c_n)^T \in \mathbb{R}^n, b=(b_1,...,b_r)^T \in \mathbb{R}^r, A \in \mathcal{M}_{r,n}(\mathbb{R})$.

In [9]:
import numpy as np

def FM1(A,b):
    """
    Concatène verticalement les matrices A et -b
    """
    A=np.array(A,dtype=float)
    b=np.array(b)
    return np.column_stack((A,-b))

In [10]:
def FM2(B):
    """
    Retourne la matrice réduite associée à B
    """
    nb_lignes=np.shape(B)[0]
    for i in range(nb_lignes):
        if B[i,0]!=0 and abs(B[i,0])!=1:
            B[i] /= abs(B[i,0])
    return B

In [11]:
def FM3(C):
    """
    Entrée : Une matrice C_pxq réduite
    Sortie : La projection sur la première variable du système
    """
    p,q = np.shape(C)
    E = np.array([]).reshape(0,q-1)
    G = np.array([]).reshape(0,q-1)
    D = np.array([]).reshape(0,q-1)
    for i in range(p):
        ligne = np.copy(C[i][1:]).reshape(1,q-1)
        if C[i][0] > 0:
            D = np.concatenate((D,-ligne))
        elif C[i][0] < 0:
            G = np.concatenate((G,ligne))
        else:
            E = np.concatenate((E,ligne))
    for g in G:
        for d in D:
            E = np.concatenate((E,(g-d).reshape(1,q-1)))
    
    return E

La fonction FM1 a pour but de réécrire les contraintes du problème $Ax \leq b$ en $Cx \leq 0$ où $C$ est la concaténation des colonnes de $A$ et de $-b$. On réduit ensuite la matrice C avec la fonction FM2, c'est-à-dire que l'on réécrit les inégalités de sorte que les coefficients de la première colonne soient égaux à 1,-1 ou 0. Enfin, la fonction FM3 utilise la méthode de Fourier-Motzkin pour projeter le polyèdre $Cx \leq 0$ sur la première variable $x_1$ du système.

On va aussi avoir besoin d'une fonction déterminant les formes affines bornant la première variable du système $Cx \leq 0 $

In [12]:
def bornes(A,nb_variables):
    """
    Entrée : La matrice A réduite définissant un polyèdre et le nombre de variables du système complet

    Sortie : L'intervalle [max(formes affines), min(formes affines)] bornant la première variable du système de contraintes Ax<=0
    sous la forme d'une liste de listes de coefficients de formes affines
    """
    output=[[],[]]
    nb_lignes,q=np.shape(A)[0],np.shape(A)[1]-1
    #q est le nombre de variables du système courant et nb_variables est le nombre de variables du système complet

    for i in range(nb_lignes):
        if A[i,0]>0: # x1 + forme affine < 0
            forme_affine = [0 for k in range(nb_variables-q+1)] + list(-A[i,1::])
            output[1].append(forme_affine)

        elif A[i,0]<0: # -x1 + forme affine < 0
            forme_affine = [0 for k in range(nb_variables-q+1)] + list(A[i,1::])
            output[0].append(forme_affine)

    #Dans le cas où la variable est non bornée
    if output[0]==[]:
        output[0]=[[0 for i in range(nb_variables)] + [-1e16]]
    if output[1]==[]:
        output[1]=[[0 for i in range(nb_variables)] + [1e16]]

    return output

 retourne une liste des formes affines bornant la première variable du système. Ainsi, output\[0] est la liste des formes affines $f_i(x)=a_{i0} + \sum_ja_{ij}x_j$ tq $x_1 \geq f_i(x)$ et output\[1] est la liste des formes affines $g_i(x)=b_{i0} + \sum_jb_{ij}x_j$ tq $x_1 \leq f_i(x)$.

Plus précisément, output\[0] = \[ \[$a_{11}, ..., a_{1n}, a_{10}$], ...] et output\[1] = \[ \[$b_{11}, ..., b_{1n}, b_{10}$], ...].

Par exemple, si on a uniquement la contrainte $x \leq y + z$, la fonction retourne \[ \[0,0,0,-1e16], \[0,1,1,0] ]


Enfin, pour minimiser la fonction $f$ sur le polyèdre $Ax \leq 0$, on effectue un changement de variables avec la fonction suivante :

In [13]:
def ChangementDeVariable(A,c,i):
    """
    Changement de variable u = c1x1 + ... + cqxq et x_k = x_k pour k!=i
    """
    tmp=np.copy(A)
    nb_lignes, nb_variables = np.shape(A)[0], np.shape(A)[1]-1

    for k in range(nb_lignes):
        if A[k,i]!=0:
            for j in range(nb_variables):
                #La constante n'est pas affectée par le changement de variable, on n'a donc pas besoin d'aller jusqu'à la dernière colonne
                if j==i:
                    tmp[k,j]=A[k,i]/c[i]
                else:
                    tmp[k,j]=A[k,j]-A[k,i]*c[j]/c[i]
    return tmp

Si l'on note f(x) la fonction à minimiser, cette dernière fonction préliminaire effectue le changement de variable 
\begin{cases}
    u=f(x) \\
    x_k = x_k \quad \text{pour $k \neq i$}
\end{cases}
pour un $i$ donné. Par exemple, dans le cas à $3$ variables $x_1$, $x_2$ et $x_3$, si on effectue le changement de variable 
\begin{cases}
    u=f(x)=c_1x_1 + c_2x_2 + c_3x_3, c_1 \neq 0 \\
    x_2 = x_2 \\
    x_3 = x_3
\end{cases}
alors la k-ieme contrainte du système $a_{k1}x_1 + a_{k2}x_2 + a_{k3}x_3 - b_k \leq 0$ devient $a_{k1}( \frac{1}{c_1}u - \frac{c_2}{c_1}x_2 -\frac{c_3}{c_1}x_3) + a_{k2}x_2 + a_{k2}x_3 - b_k \leq 0$, c'est-à-dire $\frac{a_{k1}}{c_1}u + (a_{k2} - a_{k1}\frac{c_2}{c_1})x_2 + (a_{k3} - a_{k1}\frac{c_3}{c_1}x_3) - b_k \leq 0$.

Plus généralement, si on effectue le changement de variables
\begin{cases}
    u=\sum c_ix_i \\
    x_k = x_k \quad \text{pour $k \neq i$}
\end{cases}
alors le coefficient $a_{kj}$ devient :
\begin{cases}
    \frac{a_{ki}}{c_i} \quad \text{si $j=i$} \\
    a_{kj} - a_{ki}\frac{c_j}{c_i} \quad \text{si $j \neq i$}
\end{cases}

On peut maintenant écrire la fonction principale recherchant le minimum de $f$ :

In [14]:
def FourierMotzkin(A,b,c):
    """
    Retourne une solution d'un problème de minimisation sous forme standard avec :
        A et b définissant le polyèdre de contraintes Ax<=b
        c définissant les coefficients de la fonction affine c1x1+...+cnxn
        x_i>=0 pour tout i
    """
    #Concaténation et réduction des matrices A et b
    A = FM1(A,b)
    A = FM2(A)

    #nombre de variables
    q=len(c)

    #ajout des contraintes x_j>0 à la matrice A des contraintes linéaires
    ones=-1*np.eye(q)
    ones=np.column_stack((ones,[0 for i in range(q)]))
    A=np.row_stack((A,ones))

    #changement de variable u=f(x,y,z)
    var_changee=0
    while var_changee<q:
        if c[var_changee]!=0:
            A=ChangementDeVariable(A,c,var_changee)
            break
        var_changee+=1

    #On place la colonne correspondant à la variable u en avant-dernière colonne
    tmp = np.copy(A)
    A[:,var_changee], A[:,-2] = A[:,-2], tmp[:,var_changee]
    A = FM2(A)

    ###Descente
    BORNES=[]
    for i in range(q):
        BORNES.append(bornes(A,q))
        A=FM2(FM3(A))
    
    ###Remontée
    bornes_inf = [0 for i in range(q)]

    #On regarde dans un premier temps le minimum de u
    bornes_courantes = BORNES[q-1]
    min_possibles = bornes_courantes[0]

    #Au début de la remontée, u est minoré par des formes affines constantes. 
    #Par exemple, si on a effectué le changement de variable u=x, y=y et z=z, cela donne :
    #       u <= max ( 0*z + 0*y + 0*u + c_i) et u >=  min ( 0*z + 0*y + 0*u + c_j)

    #On obtient donc directement l'inf de u, qui est le minimum de la fonction u=f(x,y,z), il nous reste à obtenir les points pour lesquels ce minimum est atteint
    inf = max( [min_possibles[i][q] for i in range(len(min_possibles))] )
    min_f = inf
    bornes_inf[q-1]=inf

    #On continue la remontée
    for k in range(q-2,-1,-1):
        bornes_courantes = BORNES[k]

        #Si le coefficient devant x_k est positif, on prend le minimum de x_k sur P, sinon on prend son maximum
        if c[k]>0:
            extrema_possibles = bornes_courantes[0]
            inf = max( [  sum(   [ bornes_inf[j] * extrema_possibles[i][j] for j in range(q) ]  ) + extrema_possibles[i][q] for i in range(len(extrema_possibles))  ] )
        else :
            extrema_possibles = bornes_courantes[1]
            inf = min( [  sum(   [ bornes_inf[j] * extrema_possibles[i][j] for j in range(q) ]  ) + extrema_possibles[i][q] for i in range(len(extrema_possibles))  ] )
        #extrema_possibles[i][q] correspondant à la constante apparaissant dans la forme affine bornant la variable
        
        bornes_inf[k] = inf

    #On a le point de minimum de toutes les variables exceptée x_{var_changee}, on peut cependant facilement le déduire des autres (cf texte),
    # à condition d'intervertir les coefficients des variables que l'on a intervertit :
    c_aux = list(np.copy(c))
    c_aux[var_changee] = c[q-1]
    c_aux[q-1] = c[var_changee]
    min_var_changee = ( bornes_inf[q-1] - sum ( [ c_aux[i]*bornes_inf[i] for i in range(q-1)]) ) / c[var_changee] 

    #Sans oublier que l'on avait échangé les places des variables u / x_{var_changee} et x_q
    #On a en effet une liste bornes_inf = [min(x_1) , ... , min(x_q) , ... , min(x_{var_changee})], on la remet donc dans l'ordre de départ
    bornes_inf[q-1] = bornes_inf[var_changee]
    bornes_inf[var_changee] = min_var_changee

    return {'optimum':min_f, 'x':bornes_inf}

La matrice
\begin{equation*}
C = 
\begin{pmatrix}
a_{11} & ... & a_{1n} & -b_1 \\
... & ... & ... & ... \\
a_{r1} & ... & a_{rn} & -b_r
\end{pmatrix}
\end{equation*}

décrit le polyèdre des contraintes linéaires du problème. On y ajoute dans un premier temps l'ensemble des contraintes $x_i \geq 0$ pour tout $i$. On effectue ensuite le changement de variable 
\begin{cases}
    u=f(x) \\
    x_k = x_k \quad \text{pour $k \neq i$}
\end{cases}
où i est le plus petit $l$ pour lequel le coefficient $c_l$ n'est pas nulle. Enfin, on place la colonne correspondant à la nouvelle variable $u$ en avant-dernière colonne et celle correspondant à la dernière variable en première colonne, cela nous permet de projeter le polyèdre sur la variable $u$ en dernier lieu en utilisant la fonction FM3.

La phase de descente consiste en des projections successives et en un stockage des coefficients des formes affines bornant les variables dans une liste BORNES où BORNES\[$k-1$] est une liste de deux listes (une pour les minorants et une pour les majorants de $x_k$).

La phase de remontée consiste ensuite à déterminer la valeur minimale (resp. maximale) des majorants (resp. des minorants) pour chaque variable.

In [15]:
#Exemples 

A=[[-1.,0,0],[1,0,0],[0,1,0],[0,0,1]]
b=[-0.5,1,1,4]
print(FourierMotzkin(A,b,[1,-1,1]))

A=[[-5,2,5]]
b=[6]
print(FourierMotzkin(A,b,[1,1,1]))

{'optimum': -0.5, 'x': [0.5, 1.0, 0.0]}
{'optimum': 0.0, 'x': [0.0, 0.0, 0.0]}
